In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

In [ ]:
from contracts.governance_contract import GovernanceContract
locked_token = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].locked_token
contract: GovernanceContract
contract = context.get_contracts(config.GOVERNANCES)[0]
print(f'Governance payment token: {locked_token}')
print(f'Governance contract: {contract.address}')
print(f'Fee token: {contract.fee_token}')

In [ ]:
# only for contract overrides
contract.address = "erd1qqqqqqqqqqqqqpgqk4tchxyrzt5pzunl6qhcnl705yrnplms0n4sa52jkw"

In [ ]:
contract = GovernanceContract(locked_token)
tx_hash, address = contract.contract_deploy(context.deployer_account, context.network_provider.proxy, config.GOVERNANCE_BYTECODE_PATH, 
                         [1000000, 3000000000000000000000000, 5000, 20, 50, 5000, 
                          context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address,
                          context.get_fee_collector_contract(0).address])
context.network_provider.check_deploy_tx_status(tx_hash, address, "Deploy Governance Contract")
contract.address = address

In [ ]:
contract.fee_token = context.get_contracts(config.EGLD_WRAPS)[0].wrapped_token
tx_hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.GOVERNANCE_BYTECODE_PATH, 
                         [0, 1000, 5000, 10, 20, 10000, 
                          context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address,
                          context.get_fee_collector_contract(0).address])
context.network_provider.check_deploy_tx_status(tx_hash, address, "Upgrade Governance Contract")

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

Contract modification configs

In [ ]:
contract.set_quorum_percentage(context.deployer_account, context.network_provider.proxy, [1000])

In [ ]:
contract.set_voting_delay(context.deployer_account, context.network_provider.proxy, [50])

In [ ]:
contract.set_voting_period(context.deployer_account, context.network_provider.proxy, [14400])

PROPOSE!

Energy Governance Message standard: {"title":"title","shortDescription":"shortDescription","strapiId":3, "version":1}

In [ ]:
from utils.utils_tx import ESDTToken
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

proposer = Account(pem_file="~/Documents/sh1.pem")
proposer.sync_nonce(context.network_provider.proxy)

fee = 1000000000000000000000000000
used_nonce = 0
found = False
tokens_in_account = get_all_token_nonces_details_for_account(contract.fee_token, proposer.address.bech32(), context.network_provider.proxy)
for token in tokens_in_account:
    print(token)
    if int(token['balance']) > fee:
        used_nonce = token['nonce']
        found = True
        break

if not found:
    raise Exception("Not enough token balance")
payment = ESDTToken(contract.fee_token, used_nonce, fee)

title = "xExchange Team Participation at xDay Satellite Event in Munich"
short_description = "This proposal invites the community to vote on xExchange’s attendance at the xDay Satellite event in Munich on October 30. The team aims to build partnerships and gain insights on Web3 innovations."
strapi_id = 7
message = '{"title":"'+ title + '","shortDescription":"' + short_description + '","strapiId":' + str(strapi_id) + ', "version":1}'

contract.propose(proposer, context.network_provider.proxy, [[payment], message])

CANCEL

In [ ]:
contract.cancel(proposer, context.network_provider.proxy, [4])

WITHDRAW

In [ ]:
from utils.utils_chain import Account

proposer = Account(pem_file=config.DEFAULT_ACCOUNTS)
proposer.sync_nonce(context.network_provider.proxy)

contract.withdraw_deposit(proposer, context.network_provider.proxy, [2])

VOTE

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address

voters = [
    'erd1x5v0kdnerkqk2tp5weqdaxersykgtcjqatz4x3l2e7kmmjs8e2xskcwstt',
    'erd1d5na7497xqz322uk4kadkjncxtv9tjg34jgl6867nhhqx5j9t74s6mv4c5',
    'erd16pe79ay2m6g7ap9vpshvvqaqc3ayplwuqw0rkwj2gqvv0egn0wjsyzvjwd'
]

voter_address = voters[1]

voter = Account(pem_file=config.DEFAULT_ACCOUNTS)
voter.address = Address(voter_address)
voter.sync_nonce(context.network_provider.proxy)

contract.vote(voter, context.network_provider.proxy, [2, 3])

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address
from random import randint
from time import sleep
from multiversx_sdk import ApiNetworkProvider
from multiversx_sdk.network_providers.interface import IPagination

class Pagination(IPagination):
    def __init__(self, start: int, size: int) -> None:
        self.start = start
        self.size = size

    def get_start(self) -> int:
        return self.start

    def get_size(self) -> int:
        return self.size

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
transactions = mainnet_api.get_transactions(Address(context.get_fee_collector_contract(0).address), Pagination(0, 100))
print(f'Number of voters: {len(transactions)}')
for tx in transactions:
    voters.append(tx.sender)

voter = Account(pem_file=config.DEFAULT_ACCOUNTS)
for voter_address in voters:
    voter.address = voter_address
    voter.sync_nonce(context.network_provider.proxy)

    contract.vote(voter, context.network_provider.proxy, [6, randint(0, 3)])
    sleep(1)